## DATA PREPROCESSING

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("category1.xlsx",sheet_name= [0,1,2,3,4,5,6,7])

In [3]:
allUserData = df[3]
userData = [df[3], df[4], df[5], df[6], df[7]]

for i in userData[1:]:
    allUserData = allUserData.append(i)

In [4]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','location_name','location_category','my_place','location_id','place_id'], 1)

In [5]:
def tomins(ts):
    a = ts.hour*60 + ts.minute
    return a

allUserData['start_time'] = allUserData['start_time'].apply(tomins)
allUserData['end_time'] = allUserData['end_time'].apply(tomins)
allUserData['duration'] = allUserData['duration'].apply(tomins)

allUserData.tail()

,user_id,weekday,start_time,end_time,duration,category,category_id,location
69,5,5,0,45,45,home,2,1
70,5,5,45,470,425,sleep,3,1
71,5,5,470,490,20,food,1,1
72,5,5,490,565,75,commute,17,99
73,5,5,565,810,245,work,6,3


In [6]:
cols = allUserData.columns
for col in cols:
    print(col)

user_id
weekday
start_time
end_time
duration
category
category_id
location


## Random Forest Model

In [7]:
train_df = allUserData[allUserData['weekday'] < 4]
test_df = allUserData[allUserData['weekday'] >= 4]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 193
Number of testing data is : 246


In [8]:
# without diff
X_train = train_df.iloc[:, np.r_[0:4, 7]].values
y_train = train_df.iloc[:, 6].values

X_test = test_df.iloc[:, np.r_[0:4, 7]].values
y_test = test_df.iloc[:, 6].values

In [9]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [10]:
clf.score(X_test, y_test)

0.8008130081300813

In [11]:
imp = clf.feature_importances_
imp

array([0.07474598, 0.03433832, 0.23517535, 0.21636921, 0.43937114])

In [19]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.59      0.67      0.62        30
           2       0.85      0.99      0.91        72
           3       1.00      0.82      0.90        33
           4       1.00      0.75      0.86         4
           5       0.00      0.00      0.00         6
           6       0.74      1.00      0.85        14
           8       0.00      0.00      0.00         5
           9       0.50      0.11      0.18         9
          10       0.00      0.00      0.00         3
          11       0.33      0.67      0.44         3
          12       0.00      0.00      0.00         0
          13       0.50      0.40      0.44         5
          16       0.50      0.67      0.57         6
          17       0.93      0.96      0.95        55
          18       0.00      0.00      0.00         1

   micro avg       0.80      0.80      0.80       246
   macro avg       0.46      0.47      0.45       246
weighted avg       0.76   

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [20]:
print(confusion_matrix(y_test,predictions))

[[20  2  0  0  0  0  0  0  0  3  1  0  3  1  0]
 [ 0 71  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  4 27  0  0  1  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  4  0  0  0  0  0  0  0  1  0  1  0  0  0]
 [ 0  0  0  0  0 14  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  1  0  1  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  1  0  1  0  0  1  1  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  2  0  0  0  0  0  2  0  0  0]
 [ 1  1  0  0  0  0  0  0  0  0  0  0  4  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0  0  0 53  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [21]:
predictions

array([ 2,  2,  3,  2, 17,  1, 17,  1, 17,  2, 17,  1, 17,  2,  2,  2,  2,
        3,  2, 17, 13, 17,  2,  2,  3,  1,  2,  2,  2,  2,  3,  2,  2,  2,
        3,  2,  1,  1, 17,  2, 17,  4, 17,  2,  3,  2, 17,  6,  1,  6, 17,
        2, 17,  4, 17,  9,  2,  2,  3,  2, 17,  6,  1,  6, 17, 13, 17,  2,
        2,  3,  2, 17,  6,  1,  6, 17, 13,  1, 17, 17,  2,  2,  3,  2, 17,
        1,  1, 17,  1,  1, 12,  9, 17,  2,  2,  3, 17,  6, 17,  1,  6, 17,
       17,  4, 17,  2,  2,  3, 17,  6,  2,  2,  3,  1,  2,  2,  2, 13, 11,
       17,  2,  2,  3,  2,  6,  2,  2,  3,  2, 17,  6, 17,  2,  2,  3,  2,
        1,  2,  2,  2,  3, 17, 12,  6,  1, 17,  2,  2,  3,  2, 17,  6,  1,
        6, 17,  2, 11,  2,  2,  3,  2, 17,  6, 11, 17,  2,  2,  3, 16, 16,
       17,  1,  1,  1, 17,  1, 11,  2,  2,  3,  2,  2,  2,  2,  6, 11, 17,
        2,  2,  3, 16,  2, 17,  6,  1,  6, 17,  1, 17,  2,  2,  3, 16, 17,
        1, 17, 17, 17,  2,  2,  3, 16, 17,  1,  1, 17,  1, 17,  1,  2,  2,
        3, 16, 17,  1,  1